# Working inside a container
Container images provide you with minimal stuff, but you can always install new tools to work with.

Let's see if we can spin up a container and `curl https://google.com`.

## Getting inside a container

<h4><span style="color: red; font-weight: bold">TODO:</span> Open a new terminal window using File > New > Terminal from the Jupyter menu bar.</h4> 

Run the following command in your terminal:

```
docker run --interactive --tty ubuntu:latest
```

Here's some explanation:

- `--interactive` (or `-i`): attaches container's `stdin`, `stdout`
- `--tty` (or `-t`): attaches a pseudo-terminal with the current terminal

In short, it allows you interact with the container easily.

You might see something like this:

```
root@<container id>:/#
```

This represents that you're inside a container now.

## Let's try a few commands

Run the following in the container:

```
ls
```

which should output something like


```
bin   dev  home  lib32  libx32  mnt  proc  run   srv  tmp  var
boot  etc  lib   lib64  media   opt  root  sbin  sys  usr
```

No problem, right? Let's `curl` now

```
curl https://google.com
```

Which should have failed with output:
```
bash: curl: command not found
```
What? But why did it fail?

Remember, I told you that Docker Images are lightweight. But you can install `curl` using `apt-get`.

```
apt-get update && apt-get install -y curl
curl https://google.com
```

OK, since its working now, let's `curl` and save the data in a file and exit from the container.

```
mkdir mydata && cd mydata
curl https://google.com > index.html
cat index.html
exit
```

Great, we downloaded a file, made sure it's there and exited from the container.

Let's confirm that the container is still there.

In [ ]:
%%bash


Yup, its there but `EXITED`.

Also note the `COMMAND` column in `docker ps -a` output, `"/bin/bash"`. More about it later.

## Reuse `EXITED` container
If you run `docker run -it ubuntu:latest`, it will spin up another container with fresh data. This container will not have `/mydata/index.html` in it.

Kick start the older container so that you get your data back. 

Use `docker start <container id>` where the `container id` is the value for the exited container.

In [ ]:
%%bash


OK, but it didn't show anything? Because now it's running in the background, doing nothing. Let's check what is inside the container again by running `/bin/bash` with a command `cat mydata/index.html`

In [ ]:
%%bash
docker exec $(docker ps -lq) /bin/bash -c 'cat mydata/index.html'

You have _executed_ a command manually inside a container and made sure that your file is still there.

The file is there. Good.

Note, if you do `docker ps` this time, it will show the container as `Up`. That's something new.

In [ ]:
%%bash


This is because the command that you executed by `docker exec` is not the primary command that was used to start the container.

You might think that `/bin/bash` is exactly the same as primary command, but it's not the first one.

## Copy data from host to a container

Say you already have something on your host machine and you want it inside the container?

Let's create a file and copy it to the container and see if it's there.

In [ ]:
%%bash
echo "hello world" > myfile.txt

Use `docker cp` to copy the file to the container:

```
docker cp <src path> <container id>:<dst path>
```

In [ ]:
%%bash


Next, use `docker exec` to run `cat` in the container and output the contents of the `mydata/hello-world.txt` file.

In [ ]:
%%bash


## Stop the container
You can stop the container now.

In [ ]:
%%bash


It should be back to `EXITED` state. Check it using `docker ps -a`.

In [ ]:
%%bash


## Save the changes
Use `docker commit`.

What? I have heard about `git commit`, what on Earth is `docker commit`?

Every command that you executed on a running container to install new tools or copied files to it, wouldn't it be nice to save them as a new image? Why not?

Try `docker commit <container id> <image name>:<image tag>`

In [ ]:
%%bash


Yes, it's that simple. Now you can run container from this image again and again.

In [ ]:
%%bash


Great job!. Now you know how to hop inside of a container!